In [2]:
!pip install earthengine-api
!pip install geemap
!pip install pycountry_convert

import ee
import geemap.foliumap as geemap

# Inicializar la API de Earth Engine
ee.Authenticate()
ee.Initialize()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
# Definir geometría de interés (aoi)
pais = "Venezuela"
gaul = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq("ADM0_NAME", pais))
aoi = gaul.geometry()

# Definir capa de elevación ALOS
alos = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").select('DSM').mosaic().clip(aoi)

# Definir capa de cobertura de bosque y quitarle deforestación
hansen = ee.Image("UMD/hansen/global_forest_change_2020_v1_8").clip(aoi)
treecover = hansen.select("treecover2000")
lossyear = hansen.select("lossyear")
treecover_2020 = treecover.where(lossyear, 0)
cob70 = treecover_2020.updateMask(treecover_2020.gte(70)).gte(1)

# Definir capa de altura de dosel (ETH) y quitarle deforestación
eth = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1").clip(aoi)
eth_def = eth.where(lossyear, 0)
alt15 = eth_def.updateMask(eth_def.gte(15)).gte(1).reproject(crs=hansen.projection(), scale=30)

# Seleccionar elevaciones entre 1000 y 2250 m y recortar las capas cob70 y alt15
rango = alos.gte(1000).And(alos.lte(2250))
cob70_rango = cob70.updateMask(rango)
alt15_rango = alt15.updateMask(rango)

# Mostrar mapa treecover70_rango y alt15_rango
location = [aoi.centroid().getInfo()['coordinates'][1], aoi.centroid().getInfo()['coordinates'][0]]
Map = geemap.Map(location=location, zoom_start=7)
Map.addLayer(cob70_rango, {"palette": "Greens"}, "Cobertura dosel =>70 en rango")
Map.addLayer(alt15_rango, {"palette": "Blues"}, "Altura dosel =>15 en rango")
Map.addLayer(lossyear, {"min": 1, "max": 20, "palette": ["pink", "red"]}, "Pérdida por año")
#Map.addLayer(alos_filtered, {}, "rango elevaciones")
Map

In [17]:
# Calcular la multiplicación de las capas de cobertura y altura de dosel
canopy_mult = cob70_rango.multiply(alt15_rango)

# Crear una capa de enmascaramiento para los píxeles superpuestos
mask = cob70_rango.gt(0).And(alt15_rango.gt(0))

# Aplicar la capa de enmascaramiento a la capa resultante
canopy_final = canopy_mult.updateMask(mask)

# Mostrar mapa final
location = [aoi.centroid().getInfo()['coordinates'][1], aoi.centroid().getInfo()['coordinates'][0]]
Map = geemap.Map(location=location, zoom_start=7)
Map.addLayer(canopy_final, {}, "Cobertura de interés en rango")
Map

In [18]:
#Obtener código ISO3 del país
import pycountry_convert as pc

iso3 = pc.country_name_to_country_alpha3(pais, cn_name_format="default")
print(iso3)

# Importar áreas protegidas
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")
wdpa_pais = wdpa.filter(ee.Filter.And(
    ee.Filter.eq("ISO3", iso3),
    ee.Filter.inList("IUCN_CAT", ["Ia", "Ib", "II", "III", "IV", "V", "VI"])
))

wdpa_image = wdpa_pais.reduceToImage(properties=['WDPAID'], reducer=ee.Reducer.first()).unmask()

# Estilo de la capa de áreas protegidas
wdpa_style = {'color': 'white', 'fillColor': 'blue'}

# Restar los píxeles de wdpa_image a la capa canopy_final
canopy_final_no_protegida = canopy_final.subtract(wdpa_image)

# Agregar capa de canopy final sin áreas protegidas al mapa
Map = geemap.Map(location=location, zoom_start=7)
Map.addLayer(wdpa_pais, wdpa_style, 'Áreas protegidas')
Map.addLayer(canopy_final_no_protegida, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'Coberturas de interés no protegidas')
Map

VEN


In [ ]:
#Tamaño de los archivos a exportar
def get_size(image):
    pixels = image.pixelLonLat().reduceRegion(ee.Reducer.count(), image.geometry(), 30, maxPixels=1e10).get("longitude")
    size = pixels.getInfo() * 30 * 30 / 1024
    if size > 1024:
        return "{:.2f} MB".format(size / 1024)
    else:
        return "{:.2f} KB".format(size)
    
size = get_size(canopy_final)
print("Tamaño de archivo de canopy_final: {}".format(size))

size = get_size(canopy_final_no_protegida)
print("Tamaño de archivo de canopy_final_no_protegida: {}".format(size))

In [ ]:
# Exportar como GeoTIFF optimizado para la nube
task = ee.batch.Export.image.toDrive(
    image = canopy_final,
    description = "canopy_final",
    folder = "GEE",
    fileNamePrefix = "canopy_final",
    scale = 30,
    region = aoi.bounds().getInfo()["coordinates"],
    fileFormat = "GeoTIFF",
    dimensions = "256x256",
    maxPixels=1e13,
    skipEmptyTiles = True,
    formatOptions = {
        "cloudOptimized": True
    }
)

#task.start()

# Exportar como GeoTIFF optimizado para la nube
task = ee.batch.Export.image.toDrive(
    image=canopy_final_no_protegida,
    description="canopy_final_no_protegida",
    folder="GEE",
    fileNamePrefix="canopy_final_no_protegida",
    scale=30,
    region = aoi.bounds().getInfo()["coordinates"],
    fileFormat = "GeoTIFF",
    dimensions = "256x256",
    maxPixels=1e13,
    skipEmptyTiles = True,
    formatOptions = {
        "cloudOptimized": True
    }
)

#task.start()

In [ ]:
tasklist = ee.batch.Task.list()
print(tasklist)